# Bài 12: SQL 
(Structured Query Language)

## 1. Set up

### 1.1. Download PostgreSQL
- Google: "download PostgreSQL"
- Go to the download page, for example: https://www.postgresql.org/download
- Choose the link for your OS (Windows, MacOS, etc.), for example: https://www.postgresql.org/download/windows
- Download the latest installer (choose 64-bit version if you're using a 64-bit machine)

### 1.2. Install PostgreSQL
- Click on the installer you've just downloaded an follow then instruction
- Remember to choose to install `pgAdmin` as well

### 1.3. Start pgAdmin
- On the search window, type "pgadmin", and click on the right result
- Type in the password if being asked

### 1.4. Create a sample database
- On the left panel, right-click and choose `Databases > Create > Database`
- In the popup box, type in the database name, i.e. `dvdrental` in this case
- Click save

### 1.5. Download the sample data
- Download the sample data here: https://sp.postgresqltutorial.com/wp-content/uploads/2019/05/dvdrental.zip
- Unzip the data to get the `.tar` file
- Remember the location of the `.tar` file, for example: `C:\Users\tue\Downloads\dvdrental\dvdrental.tar`

### 1.6. Load sample data into `dvdrental`
- In `pgAdmin`, right-click on `dvdrental` and choose `Restore`
- Navigate to `C:\Users\tue\Downloads\dvdrental\dvdrental.tar`, select the file and choose `Restore`
- Wait for a few seconds
- Now right-click on the newly created database `dvdrental` and choose `Refresh`
- Go to and exand the sections `Schemas > Tables`, you should see the database tables were loaded

- Data table ~ file excel/csv
- Database = set of data tables
- DBMS (database management system) = set of database

### 1.7. Test the loaded database
- Righ-click on `dvdrental` database, choose `Query Tool`
- In the query editor, write some simple select queries and run to test (use `F5` to run)
- Example
```sql
select * 
from actor
limit 10
```

```sql
select count(*)
from actor
```

### 1.8. View the table ERD
- View the ERD for `dvdrental` in the link below
- ERD: https://sp.postgresqltutorial.com/wp-content/uploads/2018/03/printable-postgresql-sample-database-diagram.pdf

- ERD: entity relationship diagram

### 1.9. Install `psycopg2`
- Run: `conda install -c conda-forge psycopg2`
- If not successful, run: `pip install psycopg2`
- Open a terminal and run the command
- Wait about 1-2 minutes

### More resources
- https://www.postgresqltutorial.com/
- https://www.tutorialspoint.com/sql/index.htm
- https://www.tutorialspoint.com/postgresql/index.htm

## 2. Connect Python to `dvdrental` database

In [ ]:
import psycopg2
import pandas as pd

In [ ]:
# Create a connection
conn = psycopg2.connect(
        host="localhost",
        port=5432,
        database="dvdrental",
        user="postgres",
        password="admin123",
#         gssencmode="disable" # Important to fix a bug
    )

In [ ]:
query = "select * from film limit 5"
df = pd.read_sql(query, conn)

In [ ]:
df

## 3. Basic queries
### 3.0. Một số lưu ý chung
- SQL keyword không phân biệt hoa thường, nhưng khi match data thì có
- SQL queries có thể viết trên nhiều dòng để tăng tính sáng sủa
- Cần lưu ý count data trước khi dump về local (vì có thể data rất lớn)

### 3.1. SELECT
- Tương tự subset cột trong pandas

In [ ]:
# Lấy hết các cột, limit 3 dòng đầu
query = "select * from customer limit 3"
pd.read_sql(query, conn)

In [ ]:
# Lấy 1 số cột, limit 3 dòng đầu
query = """
SELECT 
    first_name, 
    last_name, 
    email 
FROM customer 
LIMIT 3
"""

pd.read_sql(query, conn)

In [ ]:
# Select một số cột có sẵn, tạo cột mới dùng concat string
query = """
select 
    first_name,
    last_name,
    (first_name || ' ' || last_name) as full_name,
    email
from customer
limit 3
"""
pd.read_sql(query, conn)

In [ ]:
# Inspect bảng film
query = "select * from film limit 1"
pd.read_sql(query, conn)

In [ ]:
# Lấy ra title, rental_rate, length, và tạo cột rate per minute
query =  """
select 
    title,
    rental_rate,
    length,
    (rental_rate / length) as rate_per_min
from film
limit 3
"""

pd.read_sql(query, conn)

#### Thực hành

### 3.2. ORDER BY
- Tương tự `df.sort_values()` trong pandas

In [ ]:
# Sắp xếp tăng dần by first_name, descending by last_name
query = """
select
    first_name,
    last_name
from customer
order by 
    first_name,
    last_name desc
limit 10
"""

pd.read_sql(query, conn)

In [ ]:
# Sắp xếp giảm dần theo độ dài full name
# sau đó tăng dần theo ABC của full name
query = """
select
    (first_name || ' ' || last_name) as full_name,
    length((first_name || ' ' || last_name)) as full_name_length
from customer
order by 
    full_name_length desc,
    full_name asc
limit 10
"""

pd.read_sql(query, conn)

In [ ]:
# Lấy title, rental_duration, rental_rate từ bảng film
# Sắp xếp theo thứ tự tăng dần rental_duration, giảm dần rental_rate
query = """
select
    title,
    rental_duration, 
    rental_rate
from film
order by
    rental_duration,
    rental_rate desc
limit 10
"""

pd.read_sql(query, conn)

#### Thực hành

### 3.3. DISTINCT
- Tương tự `.unique()` của pandas

In [ ]:
# Inspect film table
query = "select * from film limit 1"
pd.read_sql(query, conn)

In [ ]:
# Unique year
query = "select distinct release_year from film"
pd.read_sql(query, conn)

In [ ]:
# Unique rental_rate
query = """
select distinct 
    rental_rate
from film
order by 
    rental_rate
"""

pd.read_sql(query, conn)

In [ ]:
# Unique rental_duration
query = """
select distinct 
    rental_duration
from film
order by 
    rental_duration
"""

pd.read_sql(query, conn)

In [ ]:
# Unique combinations of rental_rate, rental_duration
query = """
select distinct 
    rental_rate, 
    rental_duration
from film
order by 
    rental_rate, 
    rental_duration
"""

pd.read_sql(query, conn)

#### Thực hành

### 3.4. WHERE
- Tương tự `.loc` của pandas để filter rows

In [ ]:
# Inspect bảng customer
pd.read_sql("select * from customer limit 1", conn)

In [ ]:
# Lọc ra các khách hàng có tên bắt đầu bởi chữ b
# Wrong way
query = """
select first_name, last_name
from customer
where first_name like 'b%'
limit 3
"""

pd.read_sql(query, conn)

In [ ]:
# Lọc ra các khách hàng có tên bắt đầu bởi chữ b
# Correct way
query = """
select first_name, last_name
from customer
where lower(first_name) like 'b%'
limit 3
"""

pd.read_sql(query, conn)

In [ ]:
# Lọc ra các khách hàng first_name có chứa chữ b
query = """
select first_name
from customer
where lower(first_name) like '%b%'
limit 5
"""

pd.read_sql(query, conn)

In [ ]:
# Lọc ra các khách hàng first_name kết thúc bởi chữ b
query = """
select first_name
from customer
where lower(first_name) like '%b'
limit 5
"""

pd.read_sql(query, conn)

In [ ]:
# Lọc ra các bộ phim cho thuê >=6 ngày
query = """
select *
from film
where rental_duration >= 6
limit 3
"""

pd.read_sql(query, conn)

In [ ]:
# Chọn các cột title, length, rental_duration, rental_rate từ bảng film
# Lọc ra các dòng có rental_duration >= 6 ngày
# Và rental_rate < 3 USD
query = """
select 
    title, 
    length,
    rental_duration, 
    rental_rate
from film
where 
    (rental_duration >= 6)
    and (rental_rate < 3)
limit 5
"""

pd.read_sql(query, conn)

#### Thực hành

### 3.5. LIMIT & OFFSET

In [ ]:
# Inspect
pd.read_sql("select * from category limit 1", conn)

In [ ]:
# Limit
query = "select * from category limit 5"
pd.read_sql(query, conn)

In [ ]:
# Limit + offset
query = "select * from category limit 5 offset 3"
pd.read_sql(query, conn)

#### Thực hành

### 3.6. COUNT
- Count tương tự `.shape[0]` -> số dòng
- Count distinct tương tự `.nunique()`

In [ ]:
# Inspect
pd.read_sql("select * from film limit 1", conn)

In [ ]:
# Đếm số dòng trong bảng film
query = """
select count(*) as n_movies
from film
"""

pd.read_sql(query, conn)

In [ ]:
# Có bao nhiêu distinct loại rating từ bảng film
query = """
select count (distinct rating)
from film
"""

pd.read_sql(query, conn)

In [ ]:
# Có bao nhiêu distinct rental_rate từ bảng film
query = """
select count(distinct rental_rate)
from film
"""

pd.read_sql(query, conn)

#### Thực hành

### 3.7. IN
- Tương tự `.isin()`

In [ ]:
# Inspect film
query = "select * from film limit 1"
pd.read_sql(query, conn)

In [ ]:
# Có những distinct rating nào
query = """
select distinct rating
from film
"""
pd.read_sql(query, conn)

In [ ]:
# Lọc ra tất cả các film thuộc rating R, G, và PG
# Có bao nhiêu bộ film như vậy
query = """
select count(*)
from film
where rating in ('R', 'G', 'PG')
"""
pd.read_sql(query, conn)

#### Thực hành

### 3.8. BETWEEN
- Lưu ý include cả 2 đầu

In [ ]:
# Đếm số film có rental từ 3 đến 5 đô
# Low and high are included
# Cách 1:
query = """
select count(*)
from film
where 
    (rental_rate >= 3)
    and (rental_rate <= 5)
"""

pd.read_sql(query, conn)

In [ ]:
# Đếm số film có rental từ 3 đến 5 đô
# Low and high are included
# Cách 2:
query = """
select count(*)
from film
where rental_rate between 3 and 5
"""

pd.read_sql(query, conn)

In [ ]:
# Inspect paymentt table
query = "select * from payment limit 1"

pd.read_sql(query, conn)

In [ ]:
# Lọc ra tất cả các payment trong giai đoạn 2007/02/15 to 2007/02/19
# Trường hợp này không dùng between được
query = """
select count(*)
from payment
where 
    (payment_date >= '2007-02-15 00:00:00')
    and (payment_date < '2007-02-20 00:00:00')
"""

pd.read_sql(query, conn)

In [ ]:
# Dạng gọn hơn của VD trên
query = """
select count(*)
from payment
where 
    (payment_date >= '2007-02-15 00:00:00')
    and (payment_date < '2007-02-20 00:00:00')
"""

pd.read_sql(query, conn)

#### Thực hành

### 3.9. CAST
- type casting 
- Tương tự `.astype()`

In [ ]:
# Inspect
pd.read_sql("select * from payment limit 1", conn)

In [ ]:
# Đổi amount thành số string
query = """
select 
    customer_id,
    cast(customer_id as text) as cust_id_str 
from payment
limit 5
"""

pd.read_sql(query, conn).dtypes

#### Thực hành

### 3.10. Math operations

In [ ]:
# Inspect
query = "select * from payment limit 1"
pd.read_sql(query, conn)

In [ ]:
# Round
query = """
select round(amount, 1) as amount_1
from payment 
limit 1
"""

pd.read_sql(query, conn)

In [ ]:
# Floor
query = """
select floor(amount) as amount_1
from payment 
limit 1
"""

pd.read_sql(query, conn)

In [ ]:
# Ceiling
query = """
select ceil(amount) as amount_1
from payment 
limit 1
"""

pd.read_sql(query, conn)

In [ ]:
# Nhân amount với 1000
query = """
select amount * 1000 as amount_1
from payment 
limit 1
"""

pd.read_sql(query, conn)

In [ ]:
# Nhân staff_id với customer_id
# Stupid, only for illustration purpose
query = """
select 
    staff_id,
    customer_id,
    (staff_id * customer_id) as amount_1
from payment 
limit 1
"""

pd.read_sql(query, conn)

#### Thực hành

### 3.11. Working with dates

In [ ]:
# Inspect
query = "select * from film limit 1"
pd.read_sql(query, conn)

In [ ]:
# Lấy cột title, last_update
# Tạo cột prev_date cách 10 ngày về trước
# Tạo cột next_date cách 1 tháng trở về sau
query = """
select
    title, 
    last_update,
    (last_update - interval '10 days') as prev_date,
    (last_update + interval '1 month') as next_date
from film
limit 5
"""

pd.read_sql(query, conn)

In [ ]:
# Chỉ lấy phần date (bỏ timestamp)
query = """
select
    title, 
    last_update,
    date(last_update) last_update_date
from film
limit 2
"""

pd.read_sql(query, conn)

In [ ]:
# Chỉ lấy phần ngày, tháng, năm, từ date
query = """
select
    title, 
    last_update,
    date_part('year', last_update) as year,
    date_part('month', last_update) as month,
    date_part('day', last_update) as day
from film
limit 2
"""

pd.read_sql(query, conn)

In [ ]:
# Chỉ lấy phần ngày, tháng, năm, từ date
query = """
select
    title, 
    last_update,
    date_part('year', last_update) as year,
    date_part('month', last_update) as month,
    date_part('day', last_update) as day
from film
limit 2
"""

pd.read_sql(query, conn)

- More: https://www.postgresql.org/docs/8.1/functions-datetime.html

#### Thực hành

### 3.12. LIKE with `%` and `_`

- % match any characters
- _ match 1 character

In [ ]:
# Inspect
query = """select * from customer limit 1"""
pd.read_sql(query, conn)

In [ ]:
# Khách hàng có last_name kết thúc bởi el
query = """
select 
    last_name,
    email
from customer 
where lower(last_name) like '%el'
limit 2"""
pd.read_sql(query, conn)

In [ ]:
# Khách hàng có last_name kết thúc bởi el và một ký tự bất kỳ sau đó
query = """
select 
    last_name,
    email
from customer 
where lower(last_name) like '%el_'
limit 2"""
pd.read_sql(query, conn)

#### Thực hành

### 3.13. NULL 
- Tươn tự `None`, `NaN`
    - `.isnull()`
    - `.notnulll()`

In [ ]:
# Inspect
query = "select * from staff limit 1"
pd.read_sql(query, conn)

In [ ]:
# Tìm nhân viên chưa upload ảnh profile
query = """
select *
from staff
where picture is null
limit 5
"""

pd.read_sql(query, conn)

#### Thực hành

### 3.14. Working with string
- `.strip()`

In [ ]:
query = """select * from actor limit 1"""
pd.read_sql(query, conn)

In [ ]:
# Lower first name
query = """
select
    first_name,
    lower(first_name) as first_name_lower
from actor
limit 3
"""

pd.read_sql(query, conn)

In [ ]:
# Upper first name
query = """
select
    first_name,
    lower(first_name) as first_name_lower
from actor
limit 3
"""

pd.read_sql(query, conn)

In [ ]:
# Concat tạo full name
query = """
select
    first_name,
    last_name,
    (first_name || ' ' || last_name) as full_name
from actor
limit 3
"""

pd.read_sql(query, conn)

In [ ]:
# Trim khoảng trắng
# trim, ltrim, rtrim
query = """
select
    first_name,
    trim(first_name) as trim_first_name
from actor
limit 3
"""

pd.read_sql(query, conn)

#### Thực hành